In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

# Load the dataset
data = pd.read_csv('PhishingDataset.csv')
phish = data[data.Result == 1]
legit = data[data.Result == -1]
legit_sample = legit.sample(n=6157, replace=True)
data2 = pd.concat([legit_sample, phish], axis=0)

# Feature and target selection
X = data2.drop('Result', axis=1)
y = data2['Result']

# Feature selection using canopy
canopy = RandomForestClassifier(n_estimators=100)
canopy.fit(X, y)
feature_selector = SelectFromModel(canopy, prefit=True)
X_selected = feature_selector.transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=12)

# Support Vector Machine with Hyperparameter Tuning
svm = SVC(probability=True)
svm_params = {'C': [0.01, 0.1, 1], 'gamma': [0.01, 0.1, 1], 'kernel': ['rbf']}
svm_grid = GridSearchCV(svm, svm_params, cv=5)
svm_grid.fit(X_train, y_train)
best_svm = svm_grid.best_estimator_

# Decision Tree with Hyperparameter Tuning
tree = DecisionTreeClassifier()
tree_params = {'max_depth': [10, 20, 30]}
tree_grid = GridSearchCV(tree, tree_params, cv=5)
tree_grid.fit(X_train, y_train)
best_tree = tree_grid.best_estimator_

# Logistic Regression
lrc = LogisticRegression(max_iter=250)
lrc.fit(X_train, y_train)

# Ensemble Model
ensemble = VotingClassifier(estimators=[('Logistic', lrc), ('Decision', best_tree), ('SVC', best_svm)], voting='soft')
ensemble.fit(X_train, y_train)

# Evaluate the ensemble model
acc_train_ensemble = accuracy_score(y_train, ensemble.predict(X_train))
acc_test_ensemble = accuracy_score(y_test, ensemble.predict(X_test))



# Prediction
input_string = input("Enter 30 features separated by commas: ")
input_features = input_string.split(',')
input_features_numeric = [float(feature) for feature in input_features]
input_features_reshaped = [input_features_numeric]
prediction = ensemble.predict(input_features_reshaped)

print("Prediction:", prediction)
